In [99]:
import sys
!{sys.executable} -m pip install gym



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Environment Setup

In [115]:
import gym
import numpy as np
from gym import spaces

class ThermostatEnvironment(gym.Env):
    
    def __init__(self):
        super(ThermostatEnvironment, self).__init__()
        self.day = 1
        self.time = 1
        self.booking_status = 0
        self.outside_temp = 30
        self.inside_temp = 18
        self.action_space = spaces.Discrete(3) #increase, decrease, maintain
        self.observation_space = spaces.Box(low=np.array([1,1,0,23,18]), high=np.array([5,24,1,40,25]), dtype=np.float32)
        self.set_point = 18
        self.temperature = 18
        self.max_temp = 25
        self.min_temp = 18
        self.reward = 0
        self.done = False
    
    def step(self, action):
        #action logic
        if action == "0": #decrease
            self.temperature -=1
        elif action == "2": #increase
            self.temperature +=1
        else: #maintain
            self.temperature = self.temperature

        
        # ensure within range
        if self.temperature < self.min_temp:
            self.temperature = self.min_temp
        elif self.temperature > self.max_temp:
            self.temperature = self.max_temp

        # reward logic
        # if energy consumption is high, reward is low
        # if energy consumption is low, reward is high
        # if energy consumption is optimal, reward is highest
        

    def reset(self):
        self.temperature = 18
        self.day = 1
        self.time = 1
        self.booking_status = 0
        self.outside_temperature = 18
        return self.temperature
    
    def action_space(self):
        return self.action_space

## Neural Network

In [116]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
from collections import deque

class DQN(nn.Module):
    def __init__(self, input_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_size, 25)  # 5 input features, 1 output features
        self.fc2 = nn.Linear(25,25)
        self.fc3 = nn.Linear(25, action_size) # 10 input features, 2 output features
    
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

In [119]:
#test the model
env = ThermostatEnvironment()
model = DQN(5,3)
print(model)

DQN(
  (fc1): Linear(in_features=5, out_features=25, bias=True)
  (fc2): Linear(in_features=25, out_features=25, bias=True)
  (fc3): Linear(in_features=25, out_features=3, bias=True)
)


c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


## DQN Agent

In [120]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self.build_model()


    #return action based on state
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        state = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            act_values = self.model(state)
        return np.argmax(act_values[0])
    
    # remember experience
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            state = torch.FloatTensor(state).unsqueeze(0)
            next_state = torch.FloatTensor(next_state).unsqueeze(0)
            target = reward
            if not done:
                with torch.no_grad():
                    target = (reward + self.gamma * torch.max(self.model(next_state)[0])).item()
            target_f = self.model(state)
            target_f[0][action] = target
            self.optimizer.zero_grad()
            loss = nn.MSELoss()(target_f, self.model(state))
            loss.backward()
            self.optimizer.step()
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def build_model(self):
        self.model = DQN(self.state_size, self.action_size)  # Ensure this matches your DQN class
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        return self.model
    
   

## Training Agent

In [ ]:
# 